In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
train_data = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#check null
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [4]:
#convert numeric data to range data
def divide_convert(age, num = 10) :
    return int(age/num)

train_data["Age"] = train_data["Age"].apply(lambda x : divide_convert(x))
train_data["SkinThickness"] = train_data["SkinThickness"].apply(lambda x : divide_convert(x))

train_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,3,0,33.6,0.627,5,1
1,1,85,66,2,0,26.6,0.351,3,0
2,8,183,64,0,0,23.3,0.672,3,1
3,1,89,66,2,94,28.1,0.167,2,0
4,0,137,40,3,168,43.1,2.288,3,1


In [53]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#use last 20 datas for validate
x = train_data.iloc[:-20, :-1].to_numpy()
x = torch.FloatTensor(x).to(device)
y = train_data.iloc[:-20, -1].to_numpy()
y = torch.FloatTensor(y).to(device).reshape(-1, 1)

valid_x = train_data.iloc[-20:, :-1].to_numpy()
valid_x = torch.FloatTensor(valid_x).to(device)
valid_y = train_data.iloc[-20:, -1].to_numpy()
valid_y = torch.FloatTensor(valid_y).to(device).reshape(-1, 1)

print(x.shape, y.shape)
print(valid_x.shape, valid_y.shape)

torch.Size([748, 8]) torch.Size([748, 1])
torch.Size([20, 8]) torch.Size([20, 1])


In [54]:
import torch.nn as nn
import torch.nn.functional as F

class BinaryDnnModel(nn.Module) :
    def __init__(self, features) :
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(features, features*2),
            nn.LeakyReLU(),
            nn.Linear(features*2, features),
            nn.LeakyReLU(),
            nn.Linear(features, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x) :
        return self.model(x)

model = BinaryDnnModel(x.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [55]:
n_epoch = 500
model.train()

for epoch in range(n_epoch) :
    predict = model(x)
    
    loss = F.binary_cross_entropy(predict, y)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    predict = predict >= torch.FloatTensor([0.5]).to(device)
    predict = predict.float()
    
    acc = (predict == y)
    acc = acc.sum()
    acc = acc / x.shape[0]
    
    print('Epoch : {}/{},   loss : {:.5f},    acc : {:.5f}'.format(epoch+1, n_epoch, loss.item(), acc))

Epoch : 1/500,   loss : 1.70892,    acc : 0.55481
Epoch : 2/500,   loss : 22.77833,    acc : 0.47727
Epoch : 3/500,   loss : 28.51794,    acc : 0.34492
Epoch : 4/500,   loss : 3.26666,    acc : 0.59759
Epoch : 5/500,   loss : 0.67834,    acc : 0.66310
Epoch : 6/500,   loss : 0.65696,    acc : 0.65642
Epoch : 7/500,   loss : 2.28704,    acc : 0.34492
Epoch : 8/500,   loss : 2.01410,    acc : 0.65508
Epoch : 9/500,   loss : 2.36542,    acc : 0.65508
Epoch : 10/500,   loss : 1.26757,    acc : 0.65508
Epoch : 11/500,   loss : 0.67350,    acc : 0.61898
Epoch : 12/500,   loss : 1.07750,    acc : 0.35294
Epoch : 13/500,   loss : 0.91165,    acc : 0.36497
Epoch : 14/500,   loss : 0.65153,    acc : 0.66979
Epoch : 15/500,   loss : 0.63855,    acc : 0.66310
Epoch : 16/500,   loss : 0.63597,    acc : 0.65374
Epoch : 17/500,   loss : 0.63897,    acc : 0.65508
Epoch : 18/500,   loss : 0.64423,    acc : 0.65508
Epoch : 19/500,   loss : 0.64793,    acc : 0.65508
Epoch : 20/500,   loss : 0.64783,    a

In [56]:
model.eval()

valid_acc = 0
valid_loss = 0
for i in range(20) :
    predict = model(valid_x[i])
    
    loss = F.binary_cross_entropy(predict, valid_y[i])
    valid_loss += loss.item() / 20
    
    predict = predict >= torch.FloatTensor([0.5]).to(device)
    predict = predict.float()
    
    if predict.item() == valid_y[i].item() : valid_acc += 1
    
print('acc : {0}, loss : {1:.5f}'.format(valid_acc/20, valid_loss))

acc : 0.85, loss : 0.42159
